# Problema 1 a: Bestias y espadas

In [ ]:
def save_sword(key:str, espadas:list) -> list:
    espadas[ord(key)-97] += 1
    return espadas

def derrotar_bestia(lock:str, espadas:list) -> tuple:
    i = ord(lock)-65
    if espadas[i] > 0:
        espadas[i] -= 1
        return True, espadas
    return False, espadas

def numero_espadas(mapa:str) -> bool:
    espadas = [0]*26
    buy = 0

    for i in range(0,len(mapa),2):
        i, j = i, i+1
        espadas = save_sword(mapa[i], espadas)
        derrotar, espadas = derrotar_bestia(mapa[j], espadas)
        if not derrotar: # si no puedo derrotar a la bestia compro una espada
            buy += 1
    return buy

#### Ejecución

In [ ]:
if __name__ == "__main__": # Para que no se ejecute al importar la tarea
    print(numero_espadas('yTrSkSoVaNxEmUwXaVrBrAcMlEsFxJkYlVsIwZtNiWwDsSfStGiTsIhOlWaDpGsLcTmRhNbIzZoJiRiW'))

22


# Problema 2 a: Jaime el jinete

In [ ]:
from random import randint
from itertools import count
from heapq import heappop, heappush
import math
from collections import defaultdict

La estructura para almacenar el grafo es la siguiente

In [ ]:
class Heap:
    # codigo extraido de https://docs.python.org/3/library/heapq.html
    def __init__(self):
        self.pq = []                         # list of entries arranged in a heap
        self.entry_finder = {}               # mapping of tasks to entries
        self.REMOVED = '<removed-task>'      # placeholder for a removed task
        self.counter = count()               # unique sequence count
        
    def add(self, task, priority=0):
        'Add a new task or update the priority of an existing task'
        if task in self.entry_finder:
            self.remove(task)
        count = next(self.counter)
        entry = [priority, count, task]
        self.entry_finder[task] = entry
        heappush(self.pq, entry)
    
    def remove(self, task):
        'Mark an existing task as REMOVED.  Raise KeyError if not found.'
        entry = self.entry_finder.pop(task)
        entry[-1] = self.REMOVED
        
    def pop(self):
        'Remove and return the lowest priority task. Raise KeyError if empty.'
        while self.pq:
            _, _, task = heappop(self.pq)
            if task is not self.REMOVED:
                del self.entry_finder[task]
                return task
        raise KeyError('pop from an empty priority queue')

    def lenght(self):
        return len([x for x in self.pq if x[2] != self.REMOVED])

    def contains(self, task):
        return task in self.entry_finder

    def empty(self):
        return self.lenght() == 0

    def show(self):
        lista = [(x[2][:2], x[0]) for x in self.pq if 'removed' not in x[2]]
        lista = sorted(lista, key=lambda x: x[1])
        return lista

Heurística y funciones auxiliares

In [ ]:
def h(tablero:list, i:int, j:int, p:int, q:int) -> int:
    # heuristica, distancia euclidiana + diferencia de altura
    her = int(math.sqrt(abs(i-p)**2 + abs(j-q)**2)) # int trunca el numero
    # obtengo la alturas
    a1, a2 = tablero[i][j],  tablero[p][q]
    if a1 < a2:
        her += a2 - a1
    return her

def opciones(i:int, j:int, alto:int, ancho:int) -> list:
    ret = []
    # reviso las 8 opciones de posiciones
    # si estan dentro del tablero las retorno en ret
    for p in [i-1, i, i+1]:
        if 0 <= p < alto:
            for q in [j-1, j, j+1]:
                if 0 <= q < ancho:
                    ret.append((p, q))
    return ret

def new_g(tablero:list, g:dict, i:int, j:int, x:int, y:int) -> int:
    # distancia recorrida desde el inicio hasta (x, y) 
    # mas el costo de pasar de (x, y) a (i,j)
    d = g[(x, y)] + 1 
    d += abs(tablero[i][j] - tablero[x][y]) if tablero[i][j] > tablero[x][y] else 0
    return d

def get_inf() -> int:
    return math.inf

**Métodos de solución**: Se presenta una solución con A estrella y otra con Dijkstra. Dado que se pedía ocupar A estrella, al resolverlo con Dijkstra no obtendrá todo el puntaje.

In [ ]:
def astar(tablero:list, i:int, j:int, p:int, q:int, verbosidad=False) -> tuple:
    #posicion de inicio y final
    inicio = (i, j)
    final = (p, q)

    #lista de pos por revisar
    open_list = Heap()
    #set de pos ya visitadas
    close_list = set()
    # diccionario de el mejor nodo visitado anterior a la llave
    previous = {}
    # diccionario de la menor distancia recorrida del inicio a cada nodo
    g = defaultdict(get_inf)
    # diccionario con la mejor estimacion del mejor camino pasando por este nodo
    f = defaultdict(get_inf)

    previous[inicio] = inicio
    g[inicio] = 0
    f[inicio] = h(tablero, i, j, p, q)

    alto = len(tablero)
    ancho = len(tablero[0])

    # agrego la posicion de inicio a los nodo por revisar
    open_list.add(inicio, f[inicio])

    # mientras existan nodos a revisar sigo buscando
    while not open_list.empty():
        # elijo el nodo con menor f
        i, j = open_list.pop()       
        
        close_list.add((i, j))

        # si llego al final dejo de buscar
        if (i, j) == final:
            break

        # busco todas las nuevas opciones y las agrego a la lista de abierto
        for pos in opciones(i, j, alto, ancho):
            
            a, b = pos    
            sig_g = new_g(tablero, g, a, b, i, j)
            sig_f = sig_g+h(tablero, a, b, p, q)

            # reviso si vale la pena revisar el nodo
            if f[pos] <= sig_f:
                if open_list.contains(pos):
                    continue
                if pos in close_list:
                    continue

            if pos in close_list:
                close_list.remove(pos)
            
            # setea el nodo anterior
            previous[pos] = (i,j)
            # reemplaza anterior g
            g[pos] = sig_g
            # reemplaza anterior f
            f[pos] = sig_f
            
            open_list.add(pos, f[pos])

    # reviso si logre llegar a mi destino
    if final not in previous:
        return ([], math.inf, len(close_list)) if verbosidad else ([], math.inf)
    else:
        camino = []
        actual = final
        dist_tot = g[actual]

        while inicio != previous[actual]:
            camino.insert(0, actual)
            actual = previous[actual]
        
        camino.insert(0, actual)
        camino.insert(0, inicio)
        return (camino, dist_tot, len(close_list)) if verbosidad else (camino, dist_tot)

def dijkstra(tablero:list, i:int, j:int, p:int, q:int, verbosidad=False) -> tuple:
    #posicion de inicio y final
    inicio = (i, j)
    final = (p, q)

    #lista de pos por revisar
    open_list = Heap()
    #set de pos ya visitadas
    close_list = set()

    # diccionario de el mejor nodo visitado anterior a la llave
    previous = {}
    # diccionario de la menor distancia recorrida del inicio a cada nodo
    g = defaultdict(get_inf)

    previous[inicio] = inicio
    g[inicio] = 0

    alto = len(tablero)
    ancho = len(tablero[0])

    # agrego la posicion de inicio a los nodo por revisar
    open_list.add(inicio, 1)

    # mientras existan nodos a revisar sigo buscando
    while not open_list.empty():
        # elijo el nodo con menor f
        i, j = open_list.pop()     
        close_list.add((i,j))
        # busco todas las nuevas opciones y las agrego a la lista de abierto
        for pos in opciones(i, j, alto, ancho):
            if pos in close_list:
                continue
            a, b = pos    
            sig_g = new_g(tablero, g, a, b, i, j)

            # reviso si vale la pena actualizar distancias
            if sig_g < g[pos]:
                # setea el nodo anterior
                previous[pos] = (i,j)
                # reemplaza anterior g
                g[pos] = sig_g
                open_list.add(pos, g[pos])

    camino = []
    actual = final
    dist_tot = g[actual]

    while inicio != previous[actual]:
        camino.insert(0, actual)
        actual = previous[actual]
    
    camino.insert(0, actual)
    camino.insert(0, inicio)
    return (camino, dist_tot, alto*ancho) if verbosidad else (camino, dist_tot)

In [ ]:
def ruta_de_jaime(origen, destino, mapa, metodo = "astar"):
    # El problema debiera resolverse con A estrella, pero de todas formas se muestra una solución utilizando Dijkstra también
    metodos = {"astar": astar, "dijkstra": dijkstra}
    return metodos[metodo](mapa,*origen,*destino)

#### Ejecución

In [ ]:
if __name__ == "__main__":
    origen = (0, 2)
    destino = (3, 3)
    mapa = [[23, 33, 10, 29, 28], [25, 15, 50, 38, 12], [36, 31, 21, 44, 30], [43, 3, 24, 24, 47], [19, 37, 38, 16, 25]]
    print(ruta_de_jaime(origen, destino, mapa), "con A*")
    print(ruta_de_jaime(origen, destino, mapa, "dijkstra"),"con dijkstra")

([(0, 2), (1, 1), (2, 2), (3, 3)], 17) con A*
([(0, 2), (1, 1), (2, 2), (3, 3)], 17) con dijkstra


### Adicional

Esto ya no es parte de la solución, pero sirve para visualizar el funcionamiento del algoritmo, entenderlo mejor y su comparación con Dijkstra.

In [ ]:
def seguir_camino(tablero:list, camino:list, dist:int) -> bool:
    actual = camino.pop(0)
    altura = tablero[actual[0]][actual[1]]
    total = 0
    print(actual, altura, total)
    for actual in camino:
        total += 1
        n_altura = tablero[actual[0]][actual[1]]
        if altura < n_altura:
            total += abs(n_altura-altura)
        altura = n_altura
        print(actual, altura, total)
    return total == dist

def pprint(tablero:list):
    print('-'*(len(tablero[0])*3+1))
    for line in tablero:
        print(f"|{'|'.join([str(x).zfill(2) for x in line])}|")
        print('-'*(len(line)*3+1))

if __name__ == "__main__":            
    for _ in range(500):
        # tengo los 8 movimientos
        n = 50
        tablero = [[randint(1,50) for x in range(n)] for _ in range(n)]

        i, j, p, q = randint(0, n//2), randint(0, n//2), randint(n//2+1, n-1), randint(n//2+1, n-1)
        # pprint(tablero)
        print(f'voy de {(i, j)} a {(p, q)}')
        camino1, dist1, n_vistos = astar(tablero, i, j, p, q, True)
        print('dist:', dist1, 'n_vistos:', n_vistos, 'de', n*n)
        camino2, dist2, n_vistos = dijkstra(tablero, i, j, p, q, True)
        print('dist:', dist2, 'n_vistos:', n_vistos, 'de', n*n)
        if dist1 != dist2:
            pprint(tablero)
            print(seguir_camino(tablero, camino1, dist1))
            print()
            print(seguir_camino(tablero, camino2, dist2))
            print(tablero, i, j, p, q)
            exit()

voy de (1, 9) a (44, 32)
dist: 168 n_vistos: 1977 de 2500
dist: 168 n_vistos: 2500 de 2500
voy de (4, 13) a (43, 44)
dist: 147 n_vistos: 1963 de 2500
dist: 147 n_vistos: 2500 de 2500
voy de (19, 12) a (49, 28)
dist: 115 n_vistos: 1332 de 2500
dist: 115 n_vistos: 2500 de 2500
voy de (13, 19) a (30, 27)
dist: 97 n_vistos: 631 de 2500
dist: 97 n_vistos: 2500 de 2500
voy de (8, 2) a (46, 44)
dist: 176 n_vistos: 2192 de 2500
dist: 176 n_vistos: 2500 de 2500
voy de (15, 13) a (42, 31)
dist: 99 n_vistos: 1220 de 2500
dist: 99 n_vistos: 2500 de 2500
voy de (15, 3) a (29, 36)
dist: 117 n_vistos: 1176 de 2500
dist: 117 n_vistos: 2500 de 2500
voy de (17, 20) a (36, 38)
dist: 102 n_vistos: 710 de 2500
dist: 102 n_vistos: 2500 de 2500
voy de (7, 11) a (32, 32)
dist: 92 n_vistos: 862 de 2500
dist: 92 n_vistos: 2500 de 2500
voy de (6, 13) a (42, 36)
dist: 138 n_vistos: 1891 de 2500
dist: 138 n_vistos: 2500 de 2500
voy de (6, 20) a (31, 44)
dist: 99 n_vistos: 1014 de 2500
dist: 99 n_vistos: 2500 de 25

dist: 76 n_vistos: 533 de 2500
dist: 76 n_vistos: 2500 de 2500
voy de (12, 6) a (28, 47)
dist: 151 n_vistos: 1717 de 2500
dist: 151 n_vistos: 2500 de 2500
voy de (13, 24) a (43, 49)
dist: 108 n_vistos: 1327 de 2500
dist: 108 n_vistos: 2500 de 2500
voy de (0, 24) a (27, 45)
dist: 117 n_vistos: 931 de 2500
dist: 117 n_vistos: 2500 de 2500
voy de (6, 9) a (35, 28)
dist: 130 n_vistos: 979 de 2500
dist: 130 n_vistos: 2500 de 2500
voy de (1, 2) a (26, 30)
dist: 145 n_vistos: 786 de 2500
dist: 145 n_vistos: 2500 de 2500
voy de (12, 4) a (43, 37)
dist: 132 n_vistos: 1663 de 2500
dist: 132 n_vistos: 2500 de 2500
voy de (17, 17) a (30, 30)
dist: 63 n_vistos: 297 de 2500
dist: 63 n_vistos: 2500 de 2500
voy de (8, 1) a (40, 38)
dist: 171 n_vistos: 1355 de 2500
dist: 171 n_vistos: 2500 de 2500
voy de (23, 19) a (26, 43)
dist: 80 n_vistos: 542 de 2500
dist: 80 n_vistos: 2500 de 2500
voy de (12, 18) a (36, 40)
dist: 92 n_vistos: 1047 de 2500
dist: 92 n_vistos: 2500 de 2500
voy de (13, 6) a (45, 41)
d

dist: 176 n_vistos: 2500 de 2500
voy de (23, 16) a (44, 34)
dist: 85 n_vistos: 910 de 2500
dist: 85 n_vistos: 2500 de 2500
voy de (8, 13) a (27, 49)
dist: 138 n_vistos: 1590 de 2500
dist: 138 n_vistos: 2500 de 2500
voy de (20, 21) a (49, 28)
dist: 101 n_vistos: 1110 de 2500
dist: 101 n_vistos: 2500 de 2500
voy de (12, 17) a (46, 26)
dist: 116 n_vistos: 1574 de 2500
dist: 116 n_vistos: 2500 de 2500
voy de (3, 10) a (38, 48)
dist: 196 n_vistos: 1987 de 2500
dist: 196 n_vistos: 2500 de 2500
voy de (4, 17) a (49, 42)
dist: 175 n_vistos: 2114 de 2500
dist: 175 n_vistos: 2500 de 2500
voy de (25, 9) a (42, 47)
dist: 160 n_vistos: 2031 de 2500
dist: 160 n_vistos: 2500 de 2500
voy de (4, 13) a (38, 45)
dist: 141 n_vistos: 1642 de 2500
dist: 141 n_vistos: 2500 de 2500
voy de (3, 9) a (49, 34)
dist: 181 n_vistos: 2309 de 2500
dist: 181 n_vistos: 2500 de 2500
voy de (16, 11) a (28, 41)
dist: 98 n_vistos: 1127 de 2500
dist: 98 n_vistos: 2500 de 2500
voy de (12, 20) a (46, 26)
dist: 109 n_vistos: 12

dist: 158 n_vistos: 2500 de 2500
voy de (6, 19) a (29, 33)
dist: 87 n_vistos: 795 de 2500
dist: 87 n_vistos: 2500 de 2500
voy de (19, 4) a (37, 30)
dist: 141 n_vistos: 1046 de 2500
dist: 141 n_vistos: 2500 de 2500
voy de (6, 11) a (33, 28)
dist: 77 n_vistos: 775 de 2500
dist: 77 n_vistos: 2500 de 2500
voy de (14, 3) a (37, 45)
dist: 162 n_vistos: 1837 de 2500
dist: 162 n_vistos: 2500 de 2500
voy de (0, 14) a (32, 29)
dist: 119 n_vistos: 1084 de 2500
dist: 119 n_vistos: 2500 de 2500
voy de (4, 5) a (48, 34)
dist: 157 n_vistos: 1887 de 2500
dist: 157 n_vistos: 2500 de 2500
voy de (5, 11) a (35, 30)
dist: 104 n_vistos: 836 de 2500
dist: 104 n_vistos: 2500 de 2500
voy de (5, 21) a (44, 39)
dist: 146 n_vistos: 2039 de 2500
dist: 146 n_vistos: 2500 de 2500
voy de (13, 2) a (42, 27)
dist: 143 n_vistos: 1400 de 2500
dist: 143 n_vistos: 2500 de 2500
voy de (1, 8) a (34, 38)
dist: 151 n_vistos: 1515 de 2500
dist: 151 n_vistos: 2500 de 2500
voy de (12, 21) a (33, 27)
dist: 106 n_vistos: 793 de 25

dist: 114 n_vistos: 2500 de 2500
voy de (7, 3) a (49, 38)
dist: 172 n_vistos: 2036 de 2500
dist: 172 n_vistos: 2500 de 2500
voy de (20, 12) a (32, 45)
dist: 135 n_vistos: 1559 de 2500
dist: 135 n_vistos: 2500 de 2500
voy de (9, 16) a (43, 26)
dist: 121 n_vistos: 1104 de 2500
dist: 121 n_vistos: 2500 de 2500
voy de (24, 0) a (28, 42)
dist: 183 n_vistos: 1818 de 2500
dist: 183 n_vistos: 2500 de 2500
voy de (7, 25) a (34, 44)
dist: 102 n_vistos: 852 de 2500
dist: 102 n_vistos: 2500 de 2500
voy de (24, 11) a (46, 26)
dist: 84 n_vistos: 888 de 2500
dist: 84 n_vistos: 2500 de 2500
voy de (18, 18) a (31, 47)
dist: 132 n_vistos: 1575 de 2500
dist: 132 n_vistos: 2500 de 2500
voy de (14, 4) a (38, 36)
dist: 146 n_vistos: 1293 de 2500
dist: 146 n_vistos: 2500 de 2500
voy de (23, 16) a (30, 45)
dist: 107 n_vistos: 1492 de 2500
dist: 107 n_vistos: 2500 de 2500
voy de (12, 13) a (31, 45)
dist: 105 n_vistos: 1283 de 2500
dist: 105 n_vistos: 2500 de 2500
voy de (1, 13) a (45, 26)
dist: 154 n_vistos: 1

dist: 166 n_vistos: 1513 de 2500
dist: 166 n_vistos: 2500 de 2500
voy de (21, 3) a (32, 37)
dist: 132 n_vistos: 1501 de 2500
dist: 132 n_vistos: 2500 de 2500
voy de (4, 20) a (33, 47)
dist: 136 n_vistos: 1509 de 2500
dist: 136 n_vistos: 2500 de 2500
voy de (6, 8) a (33, 41)
dist: 132 n_vistos: 1374 de 2500
dist: 132 n_vistos: 2500 de 2500
voy de (16, 24) a (36, 30)
dist: 74 n_vistos: 479 de 2500
dist: 74 n_vistos: 2500 de 2500
voy de (24, 0) a (42, 31)
dist: 124 n_vistos: 1023 de 2500
dist: 124 n_vistos: 2500 de 2500
voy de (3, 15) a (34, 30)
dist: 124 n_vistos: 1088 de 2500
dist: 124 n_vistos: 2500 de 2500
voy de (8, 6) a (47, 43)
dist: 219 n_vistos: 2282 de 2500
dist: 219 n_vistos: 2500 de 2500
voy de (1, 15) a (49, 38)
dist: 166 n_vistos: 2164 de 2500
dist: 166 n_vistos: 2500 de 2500
voy de (15, 17) a (33, 28)
dist: 71 n_vistos: 704 de 2500
dist: 71 n_vistos: 2500 de 2500
voy de (3, 4) a (49, 49)
dist: 228 n_vistos: 2441 de 2500
dist: 228 n_vistos: 2500 de 2500
voy de (7, 18) a (33,

# Problema 2 b: Encontrando a Löbel

In [ ]:
from random import sample, randint

def get_rec_info(): # Esto no es necesario, es solo para leer la base de datos desde el txt
    # esta funcion retorna una lista con tuplas de la frecuencia y la frase asociada y
    # tambien una lista de prefijos asociada, para la cual deberian poder dar una 
    # recomendacion en menos de 1.5 segundos (en colab).
    # es importante que el archivo 'info_recomendaciones.txt' este el la misma carpeta que
    # el script. En colab se pueden subir archivos con el navegador en la izquierda. 

    with open('info_recomendaciones.txt', 'r') as file:
        ret = [(int(line[0]), ' '.join(line[2:])) for line in [l.strip().split() for l in file]]
    prefijos = sample(list(set(pal[:randint(1,9*len(pal)//10)] for pal in [p[1] for p in ret])), 3000)
    return prefijos, ret

In [ ]:
class Nodo:
    def __init__(self):
        self.terminal:bool = False
        self.hoja:bool = True
        self.cantidad:int = 0
        self.frase:str = ''
        self.hijos:dict = {}

    def agregar(self, frase: str, cantidad: int, index: int=0):
        if len(frase) == index:
            self.frase = frase
            self.cantidad = cantidad
            self.terminal = True
            return

        self.hoja = False
        letra = frase[index]

        if letra not in self.hijos:
            self.hijos[letra] = Nodo()

        self.hijos[letra].agregar(frase, cantidad, index+1)

    def cant_pal(self) -> tuple:
        return (self.cantidad, self.frase)

    def buscar_recomendacion(self) -> list:
        open_queue = list(self.hijos.keys())
        recomendaciones = []
        for opcion in open_queue:
            if self.hijos[opcion].terminal:
                recomendaciones.append(self.hijos[opcion].cant_pal())
            recomendaciones.extend(self.hijos[opcion].buscar_recomendacion())

        if len(recomendaciones) > 0:
            recomendaciones = sorted(recomendaciones, key=lambda x: x[0], reverse=True)

            return recomendaciones[0:1]
        return []

    def recomendar(self, frase: str) -> str:
        if len(frase) == 0:
            opciones = self.buscar_recomendacion()
            if len(opciones) == 0:
                return ''
            return opciones[0][1]

        letra, frase = frase[0], frase[1:]

        # si no encuento la frase retorno un str vacio
        if letra not in self.hijos:
            return ''

        return self.hijos[letra].recomendar(frase) 

In [ ]:
def motor_lobel(prefijos, historico):
    recomendaciones = []
    trie = Nodo()
    for line in historico:
        cantidad, frase = line
        trie.agregar(frase, cantidad)
    for prefijo in prefijos:
        recomendaciones.append(trie.recomendar(prefijo))

    return recomendaciones # Debe ser una lista de strings

#### Ejecución

In [ ]:
if __name__ == "__main__":
    prefijos = ['is based on the ', 'to go talk to t', 'i think this ', 'somewhere in t', 'works of art and t', 'and e', 'she would have gon', 'right ', 'but i k', 'in the ability o']
    historico = get_rec_info()[1]
    print(motor_lobel(prefijos, historico))
    #print(motor_lobel(*get_rec_info())) # ejemplo con mas prefijos

['is based on the assumption', 'to go talk to them', 'i think this should be', 'somewhere in the united states', 'works of art and the', 'and each one of them', 'she would have gone on', 'right from the very beginning', 'but i know you do', 'in the ability of the']


# Problema 3 a: Traductor extraterrestre

### Solución recursiva con backtracking

In [ ]:
# i es el largo del string revisado hasta el momento
# j es el largo del patron hasta el momento
def r_solve(string:str, i:int, pat:str, j:int, dicto:dict) -> bool:
    n = len(string)
    m = len(pat)

    # Caso base: No puedo tener un string con menos caracteres que el patron restante
    if n < m:
        return False

    # revise todo el string y ocupe todo el patron, encontre una solucion
    if i == n and j == m:
        return True

    # si ya termine con el string o el patron y aun me queda del otro, esta no era la solucion
    if i == n or j == m:
        return False

    # reviso la siguiente letra del patron
    curr = pat[j]

    # si ya he visto esta letra antes, continuo con su valor asignado
    if curr in dicto:

        # obtengo el string asignado a la letra actual del patron
        s = dicto[curr]
        k = len(s)

        # obtengo los siguientes k caracteres del string, partiendo desde el indice i.
        if i + k < len(string):
            ss = string[i:i + k]
        else:
            ss = string[i:]

        # retorno falso si el substring obtenido es diferente al almacenado anteriormente
        if ss != s:
            return False

        # en caso de que si calzara sigo buscando en la siguiente letra del patron y avanzando k indices en el string
        return r_solve(string, i + k, pat, j + 1, dicto)

    # si nunca he visto el caracter del patron
    # reviso que sea de largo, desde 1, hasta la cantidad restante de caracteres en el string
    for k in range(1, n - i + 1):

        # agrego al diccionario el substring con los siguientes k caracteres del string
        dicto[curr] = string[i:i + k]

        # reviso si es una posible solucion
        if r_solve(string, i + k, pat, j + 1, dicto):
            return True

        # else: backtrack - lo elimino y reviso con un subtring de largo k+=1
        dicto.pop(curr)

    return False

In [ ]:
def traduccion(mensaje, estructura):
    respuesta = dict()
    r_solve(mensaje, 0, estructura.replace(" ",""), 0, respuesta)
    return tuple((key, value) for key,value in respuesta.items()) # Debe ser una tupla de tuplas

#### Ejecución

In [ ]:
if __name__ == "__main__":
    mensaje = 'cxtfqynzhisqrfqyfqycxt'
    estructura = 'C D G B D D C'
    print(traduccion(mensaje, estructura))

(('C', 'cxt'), ('D', 'fqy'), ('G', 'n'), ('B', 'zhisqr'))


### Solución iterativa (solo accede a la mitad del puntaje de contenidos y mecanismos)

In [ ]:
from collections import defaultdict, Counter
from itertools import product
from random import randint
from time import time
from math import ceil

In [ ]:
def match(string: list, pattern: list, largo: dict)->tuple:
    # creo un diccionario para almacenar el valor de cada llave del patron
    opciones = {}
    i = 0
    for k in pattern:
        j = i + largo[k]
        # obtengo el substring que me da el subpatron
        opc = string[i: j]
            
        #si no esta lo agrego
        if k not in opciones:
            opciones[k] = opc
        # si ya habia visto esa llave, reviso que sea la misma que vi
        else:
            if opciones[k] != opc:
                return False, None
        # sigo avanzando sobre el string
        i = j
    return (True, opciones)

def solve(string: str, pattern: str) -> dict:

    # lista de los tipos de palabras a encontrar
    keys = list(set(pattern))
    # lista con el patron
    l_pattern = list(pattern)
    # diccionario con la cantidad de recurrencias de cada uno
    cantidad = Counter(pattern)
    # largo del string
    str_len = len(string)

    # diccionario con el largo a testear de cada llave
    largo = {}
    
    # obtengo combiancion de los largos posibles para cada llave del patron, cada largo va de 1 a techo((largo_palabra - cantidad_de_otros_espacios_necesarios)/cantidad_de_veces_que_se_repite_la_llave + 1)
    for combs in product(*[list(range(1, ceil(((str_len - sum([cantidad[p] for p in keys if p != k]))/cantidad[k])+1))) for k in keys]):
        # reviso que la cantidad de posiciones sea la correcta
        if sum([cantidad[k]*c for k, c in zip(keys, combs)]) == str_len:
            # creo un diccionario con la cantidad de espacios que voy a revisar para cada llave del patron
            for k, c in zip(keys, combs):
                largo[k] = c
            # reviso
            end, values = match(string, l_pattern, largo)
            if end:
                return values
    return {}

In [ ]:
def traduccion_iter(message: str, pattern: str) -> list:
    pat = ''.join(list(estructura.replace(' ', '')))
    dicto = solve(message, pat)
    return tuple([(key, val) for key, val in dicto.items()])

#### Ejecución

In [ ]:
if __name__ == "__main__":
    mensaje = 'cxtfqynzhisqrfqyfqycxt'
    estructura = 'C D G B D D C'
    print(traduccion_iter(mensaje, estructura))

(('C', 'cxt'), ('D', 'fqy'), ('G', 'nzhisq'), ('B', 'r'))


### Adicional

Función auxiliar para generar inputs aleatorios

In [ ]:
def generador(m, M, n, N, p):
    patron = [chr(randint(0,25)+65) for _ in range(randint(m, M))]
    if randint(0, 99) < p:
        dic = {}
        for k in patron:
            dic[k] = ''.join([chr(randint(0,25)+97) for _ in range(1, randint(n, N))])
        str_final = ''.join([dic[k] for k in patron])
    else:
        str_final = ''.join([chr(randint(0,25)+97) for _ in range(1, randint(n*m, N*M))])
    
    patron = ''.join(patron)
    return str_final, patron
# string, pattern = generador(5, 10, 3, 5, 70) # ejemplo de ejecucion generador de inputs